# Kleines Script zum Überprüfen von Links in Readinglists

Dieses Programm liest Readinglists als csv-Dateien ein und überprüft den Responsestatus der in der Datei enthaltenen Links und benachrichtigt die Benutzer anschliessend per E-Mail. 

## 1. Importieren aller notwendigen Bibliotheken

In [48]:
import csv
import pandas as pd #Einlesen und erstellen der CSV-Datei
import requests #Statusabfrage für Links
from email.message import EmailMessage #Versenden der E-Mails
import smtplib
from datetime import datetime #Generieren des Datums für Report

## 2. Erstellen aller notwendigen Funktionen

In [49]:
#E-Mail-Funktion mit Report als Anhang:
#Als erstes erstelle ich eine Funktion, die eine E-Mail mit CSV-Anhang an die Ziel-E-Mail-Addresse sendet:

def send_email_alert(sender, recipient, report):
    from email.message import EmailMessage
    import smtplib
    
#header

    msg = EmailMessage()
    msg["Subject"] = "Broken Links in Reading List"
    msg['From'] = sender
    msg['To'] = recipient

#Nachricht und Anhang
    date = datetime.now().strftime('%Y-%m-%d') 
    
    msg.set_content("Dear Team, \nsome of the links are not working. Please have a look!")

    with open(report, "rb") as fp:
        msg.add_attachment(
            fp.read(), maintype= date + "_broken_links_report", subtype="csv")
    
#Verbindung

    with smtplib.SMTP('smtp.web.de', 587) as smtplib:
        smtplib.ehlo()
        smtplib.starttls()
        smtplib.login(sender, 'password')
        smtplib.send_message(msg)

In [50]:
#Funktion für eine Gute-Nachricht-E-Mail, falls alle Links in Ordnung sind: 

def all_ok(sender, recipient):

    from email.message import EmailMessage
    import smtplib

#Header
    msg = EmailMessage()
    msg['Subject'] = 'All links are ok - EOM'
    msg['From'] = sender
    msg['To'] = recipient

#Nachricht
    msg.set_content(" ")


# Verbindung
    with smtplib.SMTP('smtp.web.de', 587) as s:
        s.ehlo()
        s.starttls()
        s.login(sender, 'password')
        s.send_message(msg)

In [51]:
#Funktion, um die Ergebnisse in eine Tabelle und dann in eine CSV-Datei umzuwandeln
#und das Programm anzuweisen, diese zu verschicken:

def Create_report(report):
    
    date = datetime.now().strftime('%Y-%m-%d') #Das Datum soll im Reportnamen generiert werden
    
    df = pd.DataFrame(report)           
    reportdf = df[0].str.split(';', expand=True)            
    reportdf.columns=['Item Links', 'Error Status']
    reportdf.to_csv(date + "_broken_links_report.csv", index = False)
    
    send_email_alert('email@web.de', 'email@web.de', date + "_broken_links_report.csv")

## 3. Beginn des eigentlichen Link-Überprüfungsprogramms

In [52]:
#Die Datei wird mit Pandas eingelesen:

reading_list = pd.read_csv('Criminal_justice_readinglist.csv')

#Die Zeile mit den Links wird isoliert und alle Zeilen ohne Links entfernt:
links = reading_list["url"].dropna()

In [53]:
#Mit "request" wird Statusabfrage gestartet und Ergebnisse in einer neuen Liste gespeichert:

broken_links = []

for url in links:
    try:
        r = requests.head(url)
        r.raise_for_status()
        
    # Alle Ausnahmen werden in einer Liste gespeichert:
    except requests.exceptions.HTTPError as http_error:
        
           broken_links.append(f"{url}; {http_error}")
            
    except requests.exceptions.RequestException as other_error:
        
           broken_links.append(f"{url}; {other_error}")
            
#Falls es keine Fehlermeldungen gibt, wird die Gute-Nachricht-E-Mail verschickt: 
            
if len(broken_links) == 0:
    all_ok('email@web.de', 'email@web.de')
                                
#Gibt es Fehlermeldungen, wird die Liste in eine CSV-Datei umgewandelt und per E-Mail verschickt:

Create_report(broken_links)

